<a href="https://colab.research.google.com/github/Mishak7/Finance/blob/main/Financial_Econometrics_Seminar_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ccapm = pd.read_excel('/content/drive/MyDrive/HSE/ccapm.xlsx')
ccapm.set_index('dates', inplace = True)
ccapm

,gb,cb,d1,d10,conrat,cinst,rinst
dates,,,,,,,
1947-04-01,-0.00310,0.00062,-0.12675,0.01471,0.017045,0.016256,-0.003672
1947-07-01,-0.00869,-0.03633,0.06840,-0.01245,-0.001926,-0.025896,-0.040917
1947-10-01,-0.06751,-0.04553,0.00864,0.00978,-0.018072,-0.007701,-0.016500
1948-01-01,-0.00421,0.00344,0.03023,-0.02645,-0.001629,0.012736,0.002337
1948-04-01,-0.00054,-0.01422,0.13383,0.10463,0.005196,-0.012251,-0.025284
...,...,...,...,...,...,...,...
1986-10-01,0.04848,0.04897,0.02106,0.04778,0.001825,0.004692,0.010726
1987-01-01,-0.00478,0.00003,0.17421,0.18540,0.003077,0.007087,-0.001250
1987-04-01,-0.06592,-0.05850,0.03540,0.03122,-0.002387,0.004138,0.000566


#### OLS for portfolios

In [ ]:
import statsmodels.api as sm

Y = ccapm[['gb', 'cb', 'd1', 'd10']]
X = sm.add_constant(ccapm['cinst'])

results = []
for col in Y.columns:
    model = sm.OLS(Y[col], X).fit()
    results.append({
        'Asset': col,
        'Beta': model.params['cinst'],
        'Std_Error': model.bse['cinst'],
        'p-value': model.pvalues['cinst']
    })

results_df = pd.DataFrame(results)
results_df

,Asset,Beta,Std_Error,p-value
0,gb,-0.023617,0.425349,0.955789
1,cb,0.142933,0.415764,0.731457
2,d1,2.286146,1.104305,0.040029
3,d10,1.287939,0.625373,0.041059


#### Consumption growth rates are endogenoues as there may be reverse causality: higher return rates means more consumption.

#### IV with c(t)

In [ ]:
from statsmodels.sandbox.regression.gmm import IV2SLS
Y = ccapm[['gb', 'cb', 'd1', 'd10']].iloc[1:]
X = sm.add_constant(ccapm['cinst']).iloc[1:]
instruments = sm.add_constant(ccapm['cinst'].shift(1).dropna())

results = []
for col in Y.columns:
    model = IV2SLS(endog=Y[col], exog=X, instrument=instruments).fit()
    results.append({
        'Asset': col,
        'Beta': model.params['cinst'],
        'Std_Error': model.bse['cinst'],
        'p-value': model.pvalues['cinst'],
    })

results_iv0_df = pd.DataFrame(results)
results_iv0_df

,Asset,Beta,Std_Error,p-value
0,gb,206.480482,5810.856814,0.971699
1,cb,260.519235,7325.803194,0.971676
2,d1,290.615622,8114.681842,0.971476
3,d10,286.784802,8033.524485,0.971567


#### Extremely large SE, Betas became much larger as well, nothing is significant

In [ ]:
first_stage = sm.OLS(X['cinst'], instruments).fit()
print("First stage F-statistic:", first_stage.fvalue)

First stage F-statistic: 0.0012637425799345437


#### We see that the instrument is extremely weak (should be at least 10 to be relevant) - there is no sense in such regression, the coefficients are not correct.

#### IV with c(t) + tb

In [ ]:
Y = ccapm[['gb', 'cb', 'd1', 'd10']].iloc[1:]
X = sm.add_constant(ccapm['cinst'].iloc[1:])
instruments = sm.add_constant(ccapm[['cinst', 'gb']].shift(1).iloc[1:])

results = []
for col in Y.columns:
    model = IV2SLS(endog=Y[col], exog=X, instrument=instruments).fit()
    results.append({
        'Asset': col,
        'Beta': model.params['cinst'],
        'Std_Error': model.bse['cinst'],
        'p-value': model.pvalues['cinst'],
    })

results_iv1_df = pd.DataFrame(results)
results_iv1_df

,Asset,Beta,Std_Error,p-value
0,gb,1.451427,4.839518,0.764634
1,cb,3.706661,5.495577,0.500979
2,d1,34.414728,29.960956,0.252414
3,d10,17.308990,15.364304,0.261610


In [ ]:
first_stage = sm.OLS(X['cinst'], instruments).fit()
print("First stage F-statistic:", first_stage.fvalue)

First stage F-statistic: 0.6787734093502946


#### 2 Instruments look a bit better - although Betas are not significant as well. F-statistic is again too small to conclude the relevance of instruments.

# Question 2

In [56]:
ccapm_hs = pd.read_excel('/content/drive/MyDrive/HSE/ccapm_hs.xlsx')
ccapm_hs.set_index('datevec', inplace = True)
ccapm_hs['CRATIO_lag1'] = ccapm_hs['CRATIO'].shift(1)
ccapm_hs['VWR_lag1'] = ccapm_hs['VWR'].shift(1)
ccapm_hs['CRATIO_lead1'] = ccapm_hs['CRATIO'].shift(-1)
ccapm_hs['TBILL_lead1'] = ccapm_hs['TBILL'].shift(-1)


ccapm_hs = ccapm_hs.dropna()
ccapm_hs

,CRATIO,TBILL,VWR,CRATIO_lag1,VWR_lag1,CRATIO_lead1,TBILL_lead1
datevec,,,,,,,
1959-04-01,0.991563,0.035976,0.106118,1.010612,0.034638,1.009608,0.018062
1959-05-01,1.009608,0.018062,-0.084368,0.991563,0.106118,0.999425,-0.003741
1959-06-01,0.999425,-0.003741,0.096085,1.009608,-0.084368,0.996430,0.034679
1959-07-01,0.996430,0.034679,-0.005752,0.999425,0.096085,1.002306,-0.013782
1959-08-01,1.002306,-0.013782,-0.035698,0.996430,-0.005752,1.005838,-0.050057
...,...,...,...,...,...,...,...
1978-07-01,1.002531,0.052796,0.030705,1.003071,-0.007879,1.006409,0.031475
1978-08-01,1.006409,0.031475,0.025306,1.002531,0.030705,1.004874,-0.014601
1978-09-01,1.004874,-0.014601,0.064093,1.006409,0.025306,0.999018,-0.109154


#### There is no suitable GMM function in python - let's write it manually
#### By the way, I will not introduce SE here due to lack of time

In [57]:
from scipy.optimize import minimize

# Function which will be minimized - here we write our moment condition
def gmm_objective(params, data, instruments):
  beta, gamma = params

  u = beta*ccapm_hs['CRATIO']**(-gamma)*(1+ccapm_hs['TBILL']) - 1

  moments = np.column_stack([u*inst for inst in instruments])

  m = np.mean(moments, axis=0)

  return m @ m

In [138]:
from scipy.stats import chi2

def j_test_statistic(params, data, instruments):
    beta, gamma = params

    u = beta * (data['CRATIO'])**(-gamma) * (1 + data['TBILL']) - 1
    moments = np.column_stack([u * inst for inst in instruments])
    g_bar = np.mean(moments, axis=0)

    T = len(data)
    j_stat = T * (g_bar @ g_bar)

    critical_value = chi2.ppf(0.95, len(instruments)-len(data.columns))

    if j_stat > critical_value:
      return "Reject H0 -> problem in specification"
    else:
      return"Cannot reject H0 -> model is OK"

#### Initial instruments
#### I have chosen 'Nelder-Mead' method, because it is the only one that gives simillar to STATA code numbers
#### By the way, beta is discount factor and gamma - degree of risk aversion
#### Here, discount factor seems to be reasonable, gamma - not really consistent

In [58]:
initial_values = [1, 1]
instruments = []
instruments.append(np.ones(len(ccapm_hs)))

instruments.append(ccapm_hs['CRATIO_lag1'])

result = minimize(
        gmm_objective,
        initial_values,
        args=(ccapm_hs[['CRATIO', 'TBILL']], instruments),
        method = 'Nelder-Mead'


    )

result.x

array([0.99836724, 0.55891913])

#### Adding r_t
#### I have switched to L-BFGS-B method to account for boundaries - otherwise the algorithm gives values with no economic sense
#### Estimations seem to be pretty reasonable, although beta should be less than 1

In [144]:
initial_values = [0.98, 3.0]
instruments = []
instruments.append(np.ones(len(ccapm_hs)))

instruments.append(ccapm_hs['CRATIO_lag1'])
instruments.append(ccapm_hs['TBILL'].shift(1).fillna(0.00267))

result = minimize(
        gmm_objective,
        initial_values,
        args=(ccapm_hs[['CRATIO', 'TBILL']], instruments),
        method='L-BFGS-B',
        bounds=[(0.5, 1.2), (0.1, 10)]


    )
print(f"Beta: {result.x[0]:.6f} Gamma: {result.x[1]:.6f}")
j_test_statistic(result.x, ccapm_hs[['CRATIO', 'TBILL']], instruments)

Beta: 1.003222 Gamma: 2.999955


'Cannot reject H0 -> model is OK'

#### Adding VWR does not really affect the coefficients

In [145]:
initial_values = [0.98, 3.0]
instruments = []
instruments.append(np.ones(len(ccapm_hs)))

instruments.append(ccapm_hs['CRATIO_lag1'])
instruments.append(ccapm_hs['TBILL'].shift(1).fillna(0.00267))
instruments.append(ccapm_hs['VWR'].shift(1).fillna(.0204))

result = minimize(
        gmm_objective,
        initial_values,
        args=(ccapm_hs[['CRATIO', 'TBILL']], instruments),
        method='L-BFGS-B',
        bounds=[(0.5, 1.2), (0.1, 10)]


    )

print(f"Beta: {result.x[0]:.6f} Gamma: {result.x[1]:.6f}")
j_test_statistic(result.x, ccapm_hs[['CRATIO', 'TBILL']], instruments)

Beta: 1.003221 Gamma: 2.999955


'Cannot reject H0 -> model is OK'

#### Another instrument did not changed the results

In [146]:
initial_values = [0.98, 3.0]
instruments = []
instruments.append(np.ones(len(ccapm_hs)))

instruments.append(ccapm_hs['CRATIO_lag1'])
instruments.append(ccapm_hs['TBILL'].shift(1).fillna(0.00267))
instruments.append(ccapm_hs['VWR'].shift(1).fillna(.0204))
instruments.append(ccapm_hs['VWR'].shift(2).fillna(.0204))

result = minimize(
        gmm_objective,
        initial_values,
        args=(ccapm_hs[['CRATIO', 'TBILL']], instruments),
        method='L-BFGS-B',
        bounds=[(0.5, 1.3), (0.1, 10)]


    )

print(f"Beta: {result.x[0]:.6f} Gamma: {result.x[1]:.6f}")
j_test_statistic(result.x, ccapm_hs[['CRATIO', 'TBILL']], instruments)

Beta: 1.003222 Gamma: 2.999955


'Cannot reject H0 -> model is OK'

#### It seems that we end up with relatively stable Beta: 1.003222 Gamma: 2.999955 - all models pass the tests, results are OK

# Question 3

#### I am not sure whether I am able to do it...